In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
import json

In [3]:
conf = SparkConf().setAppName('word count').setMaster('local[3]')
sc = SparkContext(conf = conf)

In [25]:
lines = sc.textFile('./data/4300-0.txt,./data/pg*.txt')

In [26]:
stoplist = sc.textFile('./data/stopwords.txt')

In [29]:
broadcastStopList = sc.broadcast(set(stoplist.collect()))

In [30]:
broadcastStopList.value

{'a',
 'about',
 'all',
 'an',
 'and',
 'any',
 'are',
 'as',
 'at',
 'be',
 'been',
 'but',
 'by',
 'can',
 'could',
 'do',
 'for',
 'from',
 'had',
 'has',
 'he',
 'her',
 'him',
 'his',
 'if',
 'in',
 'into',
 'is',
 'it',
 'may',
 'me',
 'my',
 'of',
 'on',
 'or',
 'other',
 'our',
 'said',
 'shall',
 'she',
 'so',
 'some',
 'such',
 'than',
 'that',
 'the',
 'their',
 'them',
 'then',
 'there',
 'they',
 'this',
 'to',
 'very',
 'was',
 'we',
 'were',
 'what',
 'when',
 'which',
 'who',
 'will',
 'with',
 'would',
 'your'}

In [31]:
words = lines.flatMap(lambda line : line.split())

In [32]:
filteredWords = words.filter(lambda word : word.lower() not in broadcastStopList.value)

In [33]:
counts = filteredWords.map(lambda word : (word.lower(), 1)).reduceByKey(lambda a, b : a + b)

In [34]:
output = counts.filter(lambda x : x[1] > 2).takeOrdered(40, key=lambda x : -x[1])

In [35]:
output

[('i', 11044),
 ('not', 8141),
 ('you', 6354),
 ('have', 5146),
 ('no', 3620),
 ('one', 3498),
 ('like', 2253),
 ('more', 2087),
 ('out', 2021),
 ('up', 1831),
 ('man', 1783),
 ('now', 1579),
 ('only', 1555),
 ('must', 1523),
 ('little', 1485),
 ('those', 1447),
 ('good', 1444),
 ('should', 1417),
 ('after', 1379),
 ('great', 1358),
 ('every', 1356),
 ('first', 1318),
 ('own', 1289),
 ('did', 1271),
 ('how', 1266),
 ('see', 1251),
 ('these', 1244),
 ('men', 1233),
 ('over', 1209),
 ('where', 1205),
 ('make', 1196),
 ('upon', 1188),
 ('nor', 1181),
 ('never', 1177),
 ('much', 1167),
 ('time', 1166),
 ('said,', 1163),
 ('two', 1142),
 ('old', 1140),
 ('made', 1128)]

In [36]:
output = dict(output)

In [37]:
with open('sp2.json', 'w') as outfile:
    json.dump(output, outfile)